In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import time
import tensorflow as tf
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
import os

In [3]:
from eventcnn.datasources.davis import DavisDataset
from eventcnn.datasources.eventsource import EventSource, CachedDenseEventSource

In [4]:
source = CachedDenseEventSource("data/dense")

In [5]:
N_EVENT_SLICES = source.event_slices
BATCH_SIZE = 10
N_EVENT_ROWS = source.rows
N_EVENT_COLS = source.cols
LABEL_SCALE_FACTOR = np.float32(1.0 / np.max(np.abs(source.training()["labels"])))  # Rescale the positions to be in the range [-1, 1]

In [6]:
# Copied from the tensorflow tutorial in class
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def init_bias(shape):
    return tf.Variable(tf.zeros(shape))

In [7]:
def n_output_channels(layer):
    return layer.get_shape()[-1].value

def n_outputs(layer):
    out = layer.get_shape()[1].value
    for d in layer.get_shape()[2:]:
        out *= d.value
    return out


In [8]:
WeightBias = namedtuple("WeightBias", ["weights", "bias"])

In [12]:
input_tensor = tf.placeholder(tf.float32, [None, 
                                           N_EVENT_COLS, 
                                           N_EVENT_ROWS, 
                                           N_EVENT_SLICES, 
                                           2])
dropout_keep_prob = tf.placeholder(tf.float32)

# Layer1: 3D convolution + relu activation + max_pool
weight_vars = {"conv1": WeightBias(init_weights([3, 3, 3, 2, 6]),
                        init_bias(6))}
conv1 = tf.nn.conv3d(input_tensor, weight_vars["conv1"].weights, 
                     strides=[1,1,1,1,1], padding='SAME')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, weight_vars["conv1"].bias))
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], 
                         strides=[1, 2, 2, 2, 1], padding="SAME")

# Layer2: 3D convolution + relu activation + max_pool
weight_vars["conv2"] = WeightBias(init_weights([3,3,2, n_output_channels(pool1),6]),
                                  init_bias(6))
conv2 = tf.nn.conv3d(pool1, 
                     weight_vars["conv2"].weights, 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, weight_vars["conv2"].bias))
pool2 = tf.nn.max_pool3d(conv1, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

# Layer3: 3D convolution + relu activation + max_pool
weight_vars["conv3"] = WeightBias(init_weights([3,3,2, n_output_channels(pool2),6]),
                                  init_bias(6))
conv3 = tf.nn.conv3d(pool2, 
                     weight_vars["conv3"].weights, 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv3 = tf.nn.relu(tf.nn.bias_add(conv3, weight_vars["conv3"].bias))
pool3 = tf.nn.max_pool3d(conv3, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

# Layer4: 3D convolution + relu activation + max_pool
weight_vars["conv4"] = WeightBias(init_weights([3,3,2, n_output_channels(pool2),6]),
                                  init_bias(6))
conv4 = tf.nn.conv3d(pool3, 
                     weight_vars["conv4"].weights, 
                     strides=[1, 1, 1, 1, 1], padding="SAME")
conv4 = tf.nn.relu(tf.nn.bias_add(conv4, weight_vars["conv4"].bias))
pool4 = tf.nn.max_pool3d(conv4, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding="SAME")

# Layer5: Fully connected + relu + dropout
weight_vars["fc1"] = WeightBias(init_weights([n_outputs(pool4), 32]),
                                init_bias(32))
fc1 = tf.reshape(pool4, [-1, n_outputs(pool4)])
fc1 = tf.add(tf.matmul(fc1, weight_vars["fc1"].weights), weight_vars["fc1"].bias)
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, dropout_keep_prob)

# Layer4: Fully connected + relu  + dropout
#weight_vars["fc4"] = WeightBias(init_weights([n_outputs(fc3), 32]),
#                                init_bias(32))
#fc4 = tf.reshape(fc3, [-1, n_outputs(fc3)])
#fc4 = tf.add(tf.matmul(fc4, weight_vars["fc4"].weights), weight_vars["fc4"].bias)
#fc4 = tf.nn.relu(fc4)
#fc4 = tf.nn.dropout(fc4, dropout_keep_prob)

# Output layer
weight_vars["out"] = WeightBias(init_weights([n_outputs(fc1), 3]),
                                init_bias(3))
out = tf.add(tf.matmul(fc1, weight_vars["out"].weights), weight_vars["out"].bias)

y = tf.placeholder(tf.float32, [None, 3])

loss = tf.nn.l2_loss(tf.sub(out, tf.mul(LABEL_SCALE_FACTOR, y)))
avg_loss = tf.reduce_mean(loss)
train_optimizer = tf.train.AdamOptimizer(learning_rate=0.0005).minimize(loss)

init = tf.initialize_all_variables()

saver = tf.train.Saver()

In [13]:
sess = tf.Session()
sess.run(init)
os.makedirs("models", exist_ok=True)

In [14]:
TRAINING_STEPS = 50
#BATCH_SIZE = 50
for step in range(TRAINING_STEPS):
    for i in range(0, source.num_training - BATCH_SIZE, BATCH_SIZE):
        # print("training step: {:d}, batch starting index: {:d}".format(step, i))
        events = source.training()["events"][i:(i+BATCH_SIZE),:,...,:]
        labels = source.training()["labels"][i:(i+BATCH_SIZE),:]
        avg_loss_train, opt_res = sess.run([avg_loss, train_optimizer], feed_dict={input_tensor: events,
                                             y: labels,
                                             dropout_keep_prob: 0.5})

    events_test = source.testing()["events"][1:(1+BATCH_SIZE),:,...,:] # shrink the size to batchsize, otherwise out of resources
    labels_test = source.testing()["labels"][1:(1+BATCH_SIZE),:]
    avg_loss_test = sess.run(avg_loss, feed_dict={input_tensor: events_test,
                                        y: labels_test,
                                        dropout_keep_prob: 1.0})
    print("training step: {:d}, average loss test: {:f}, average loss train: {:f}".format(step, avg_loss_test, avg_loss_train))
    # saver.save(sess, "models/model.ckpt")
    print("model saved")
avg_loss_test

training step: 0, average loss test: 0.147020, average loss train: 0.222167
model saved
training step: 1, average loss test: 0.112507, average loss train: 0.152313
model saved
training step: 2, average loss test: 0.113772, average loss train: 0.108903
model saved
training step: 3, average loss test: 0.100352, average loss train: 0.140276
model saved
training step: 4, average loss test: 0.099820, average loss train: 0.135724
model saved
training step: 5, average loss test: 0.090190, average loss train: 0.154710
model saved
training step: 6, average loss test: 0.089551, average loss train: 0.144379
model saved
training step: 7, average loss test: 0.073039, average loss train: 0.151977
model saved
training step: 8, average loss test: 0.089933, average loss train: 0.125860
model saved
training step: 9, average loss test: 0.071816, average loss train: 0.140721
model saved
training step: 10, average loss test: 0.057850, average loss train: 0.117874
model saved
training step: 11, average loss

KeyboardInterrupt: 

In [ ]:
w = sess.run(weight_vars["conv1"].weights)

In [ ]:
w.shape

In [ ]:
def get_color(data): return cm.coolwarm((data + 0.1) / 0.2)

def plot_weights_3d(ax, data): 
    for z in range(3): 
        X, Y = np.meshgrid(np.arange(0, 4), np.arange(0, 4)) 
        Z = z * np.ones_like(X) 
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, facecolors=get_color(data[:,:,z]), shade=False)

fig = figure() 
ax = fig.add_subplot(121, projection='3d') 
plot_weights_3d(ax, w[:,:,:,0,2]) 
title("Negative polarity") 
ax = fig.add_subplot(122, projection='3d') 
plot_weights_3d(ax, w[:,:,:,1,2]) 
title("Positive polarity")


In [ ]:
labels_test

def get_color(data):
    return cm.coolwarm((data + 0.1) / 0.2)


def plot_weights_3d(ax, data):
    for z in range(3):
        X, Y = np.meshgrid(np.arange(0, 4), np.arange(0, 4))
        Z = z * np.ones_like(X)
        ax.plot_surface(X, Y, Z, rstride=1, cstride=1, facecolors=get_color(data[:,:,z]), shade=False)

fig = figure()
ax = fig.add_subplot(121, projection='3d')
plot_weights_3d(ax, w[:,:,:,0,2])
title("Negative polarity")
ax = fig.add_subplot(122, projection='3d')
plot_weights_3d(ax, w[:,:,:,1,2])
title("Positive polarity")

events_test = source.testing()["events"]
labels_test = source.testing()["labels"]
# Restore variables from disk.
#saver.restore(sess, "models/model.ckpt")
print("Model restored.")
for i in range(300, 310):
    predicted = sess.run(out, feed_dict={input_tensor: events_test[i:i+1,:,...,:],
                              dropout_keep_prob: 1.0})
    expected = labels_test[i,:] * LABEL_SCALE_FACTOR
    print(predicted, expected)
    
    

In [15]:
for i in range(1, 10):
    predicted = sess.run(out, feed_dict={input_tensor: events_test[i:i+1,:,...,:],
                              dropout_keep_prob: 1.0})
    expected = labels_test[i,:] * LABEL_SCALE_FACTOR
    print(predicted, expected)
    
    



[[-0.01373429 -0.08557818  0.00990946]] [ 0.03254471 -0.17973549 -0.01043114]
[[-0.0092594  -0.09449475  0.01315229]] [-0.03443561 -0.13066569  0.01585138]
[[-0.01538954 -0.07466185  0.00981065]] [-0.03000929 -0.11348017  0.02175278]
[[-0.00726168 -0.06361941  0.01344946]] [-0.00937349 -0.08947107  0.00872576]
[[ 0.02000112  0.06324282  0.01978746]] [ 0.01009532  0.1846905  -0.01796437]
[[ 0.02153623  0.08554859  0.03101539]] [ 0.00762053  0.1715054  -0.01500311]
[[ 0.02090516  0.09461404  0.02196684]] [ 0.00961326  0.19209786 -0.01129032]
[[ 0.01245524  0.11859871  0.00432916]] [-0.02017883  0.20260449 -0.03535776]
[[ 0.00809861  0.14638945  0.02750075]] [-0.02410606  0.21354284 -0.03364459]


In [17]:
# Evaluate some of the trajectory
predicted = sess.run(out, feed_dict={input_tensor: events_test[1:10,:,...,:],
                              dropout_keep_prob: 1.0})
expected = labels_test[1:10,:] * LABEL_SCALE_FACTOR
print(np.sum(expected, axis=0), np.sum(predicted, axis=0))

[-0.05822946  0.45108868 -0.07736137] [ 0.03735141  0.19003931  0.15092155]
